In [100]:
import os
os.environ["PYSPARK_PYTHON"]="/home/hadoop/anaconda3/bin/python"
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructField, StructField, StructType, StringType, LongType
from pyspark.sql.functions import col, column, expr, lit

In [33]:
# DataFrame 创建实例
spark = SparkSession.builder.getOrCreate()
df = spark.read.json("file:///usr/local/spark/examples/src/main/resources/people.json")
# df = spark.read.format("json").load("file:///usr/local/spark/examples/src/main/resources/people.json")
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [23]:
# 打印模式信息
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [25]:
# 利用行分隔的 JSON 半结构化性质来定义这个结构
spark.read.format("json").load("file:///usr/local/spark/examples/src/main/resources/people.json").schema
# 一个模式是由许多字段构成的 StructType，这些字段即为StructField，具有名称、类型、布尔标志，且用户可以指定与该列关联的元数据(metadata)

StructType(List(StructField(age,LongType,true),StructField(name,StringType,true)))

In [29]:
# 为一个DataFrame 创建并指定模式
myManualSchema = StructType([
    StructField("age", LongType(), True),
    StructField("name", StringType(), True)
])
df1 = spark.read.format("json").schema(myManualSchema)\
    .load("file:///usr/local/spark/examples/src/main/resources/people.json")
df1.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [32]:
# 构造列
col("someColumnName")
column("someColumnName")

Column<b'someColumnName'>

In [36]:
# 访问 DataFrame 的列
spark.read.format("json")\
    .load("file:///usr/local/spark/examples/src/main/resources/people.json").columns

['age', 'name']

In [37]:
# 查看一行
df.first()

Row(age=None, name='Michael')

In [38]:
# 创建行
myRow = Row("Hello", None, 1, False)
myRow[0], myRow[1]

('Hello', None)

In [ ]:
# DF转换操作

In [41]:
# 创建 DataFrame
df3 = spark.read.format("json")\
    .load("file:///usr/local/spark/examples/src/main/resources/people.json")
df3.createOrReplaceTempView("dfTable")
df3.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [42]:
# 通过获取一组行并将其转换为 DataFrame 以创建一个临时的 DataFrame
myManualSchema = StructType([
    StructField("some", StringType(), True),
    StructField("col", StringType(), True),
    StructField("names", LongType(), False)
])
myRow = Row("Hello", None, 1)
myDf = spark.createDataFrame([myRow], myManualSchema) 
myDf.show()

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



In [46]:
# select 函数
df.select("age").show()
df.select(df.age).show()
df.select(df.age + 1).show()
df.select(df.name, df.age).show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+

+---------+
|(age + 1)|
+---------+
|     null|
|       31|
|       20|
+---------+

+-------+----+
|   name| age|
+-------+----+
|Michael|null|
|   Andy|  30|
| Justin|  19|
+-------+----+



In [66]:
# expr
# 更改列名
df.select(expr("name as NAMEtest")).show()
df.select(df.name.alias("username"),df.age).show()
# 改回去
df.select(expr("name as NAMEtest").alias("name")).show()

+--------+
|NAMEtest|
+--------+
| Michael|
|    Andy|
|  Justin|
+--------+

+--------+----+
|username| age|
+--------+----+
| Michael|null|
|    Andy|  30|
|  Justin|  19|
+--------+----+

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [65]:
# selectExpr
df.selectExpr("name as NAMEtest").show()
# 使用 select 语句时使用系统预定好的聚合函数
df.selectExpr("avg(age)", "count(distinct(name))").show()

+--------+
|NAMEtest|
+--------+
| Michael|
|    Andy|
|  Justin|
+--------+

+--------+--------------------+
|avg(age)|count(DISTINCT name)|
+--------+--------------------+
|    24.5|                   3|
+--------+--------------------+



In [71]:
# 转换操作成 Spark 类型(字面量)
df.select(expr("*"), lit(1).alias("One")).show()

+----+-------+---+
| age|   name|One|
+----+-------+---+
|null|Michael|  1|
|  30|   Andy|  1|
|  19| Justin|  1|
+----+-------+---+



In [125]:
# 添加列
df.withColumn("numberOne", lit(1)).show()
# 重命名列
df.withColumn("names", expr("name")).columns

+----+-------+---------+
| age|   name|numberOne|
+----+-------+---------+
|null|Michael|        1|
|  30|   Andy|        1|
|  19| Justin|        1|
+----+-------+---------+



['age', 'name', 'names']

In [76]:
# 使用 withColumnRenamed 重命名列
df.withColumnRenamed("name", "newname").columns

['age', 'newname']

In [78]:
# 删除列
df.drop("name").columns

['age']

In [87]:
# 更改列的类型(强制类型转换)
df.printSchema()
df.withColumn("age", col("age").cast("string"))

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



DataFrame[age: string, name: string]

In [90]:
# 过滤行
df.filter(df.age > 20 ).show()
df.where(df.age > 20 ).show()
df.filter(df.age > 15 ).filter(df.age<20).show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+



In [92]:
# 去重并返回DataFrame
df.select("name", "age").distinct().count()

3

In [95]:
# 随机抽样
seed = 5
withReplacement = False # 有无放回抽样
fraction = 0.5
df.sample(withReplacement, fraction, seed).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
+----+-------+



In [96]:
# 随机分割
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count()

False

In [112]:
# 行排序
df.sort(df.age.asc()).show()
df.orderBy(df.age.desc()).show()
# 多列排序
df.sort(df.age.desc(), df.name.asc()).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  19| Justin|
|  30|   Andy|
+----+-------+

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|  19| Justin|
|null|Michael|
+----+-------+



In [126]:
# limit方法
df.limit(2).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
+----+-------+



In [118]:
# 分组聚合
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+

